Snake Game

### Primeiro, importamos as bibliotecas necessárias.
Verifique no código anterior um script para instalar as dependências necessárias (OpenGL e GLFW) antes de prosseguir.

In [182]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
from glfw import *

### Inicializando janela

In [183]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(900, 900, "Cobrinha", None, None)
glfw.make_context_current(window)

### Capturando eventos de teclado e mouse

In [184]:
def key_event(window,key,scancode,action,mods):
    print('[key event] key=',key)
    print('[key event] scancode=',scancode)
    print('[key event] action=',action)
    print('[key event] mods=',mods)
    print('-------')
    
glfw.set_key_callback(window,key_event)

def mouse_event(window,button,action,mods):
    print('[mouse event] button=',button)
    print('[mouse event] action=',action)
    print('[mouse event] mods=',mods)
    print('-------')
glfw.set_mouse_button_callback(window,mouse_event)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec2.
* Usamos vec2, pois nosso programa (na CPU) irá enviar apenas duas coordenadas para plotar um ponto. Podemos mandar três coordenadas (vec3) e até mesmo quatro coordenadas (vec4).
* void main() é o ponto de entrada do nosso programa (função principal).
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4.

In [185]:
vertex_code = """
        attribute vec2 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,0.0,1.0);
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal).
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

In [186]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [187]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

### Associando nosso código-fonte aos slots solicitados

In [188]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [189]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [190]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [191]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

### Linkagem do programa

In [192]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

In [193]:
from enum import Enum

class GameObjectType(Enum):
    SNAKE, FOOD = list(range(2))

class SnakeDirections(Enum):
    UP = (0, 1)
    DOWN = (0, -1)
    RIGHT = (1, 0)
    LEFT = (-1, 0)

STD_SNAKE_SPEED = 0.0005
SQUARE_SCALE = 0.1
GRID_SIZE = int(2 / SQUARE_SCALE)
FRAMERATE = 10

board = [[False for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]

### Preparando dados para enviar a GPU

Nesse momento, compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Aqui, temos as classes desenhadas para gerir o funcionamento do jogo.

In [194]:
def generate_square(coord_x: int, coord_y: int):
    square = np.zeros(4, [("position", np.float32, 2)])
    square['position'] = [
        (coord_x, coord_y),
        (coord_x, coord_y + SQUARE_SCALE),
        (coord_x + SQUARE_SCALE, coord_y),
        (coord_x + SQUARE_SCALE, coord_y + SQUARE_SCALE)
    ]
    return square

In [195]:
import collections
from math import floor

def lists_are_equal(a, b):
    for first_element, second_element in a, b:
        if first_element != second_element: return False
    
    return True

class GameObject:
    def __init__(self, obj_type: GameObjectType, coordinates: list[int, int]):
        self.type = obj_type
        self.square_vertices_matrix = generate_square(coordinates[0], coordinates[1])
        self.coordinates = coordinates
    
    def get_type(self) -> GameObjectType:
        return self.type
    
class Snake:
    def __init__(self, coordinates: list[int, int]):
        self.body: list[GameObject] = [GameObject(GameObjectType.SNAKE, coordinates)]
        self.body_len = 1

        self.snake_direction: tuple[int, int] = SnakeDirections.UP
        self.head_coordinates = coordinates
        
    def update_direction(self, new_direction: tuple[int, int]):
        self.snake_direction = new_direction
    
    def eat(self, food_position: list[int, int]) -> bool:
        if lists_are_equal(self.head_coordinates, food_position):
            self.body_len += 1
            return True
        
        return False

    def update_position(self):
        # If no food has been eaten.
        if self.body_len == len(self.body):
            for i in range(len(self.body) - 1):
                self.body[i].coordinates = self.body[i + 1].coordinates
        
            self.body[self.body_len - 1] = GameObject(GameObjectType.SNAKE, self.head_coordinates)
        
        else:
            self.body.append(GameObject(GameObjectType.SNAKE, self.head_coordinates))

        self.head_coordinates = [self.head_coordinates[i] * SQUARE_SCALE for i in range(2)]

def spawn_food():
    rand_x = int(np.random.random() * 20)
    rand_y = int(np.random.random() * 20)

    count = 0
    while board[rand_x][rand_y] and count < GRID_SIZE ** 2:
        rand_y += 1
        if rand_y == GRID_SIZE:
            rand_x += 1
            rand_y = 0

            if rand_x == GRID_SIZE:
                rand_x = 0

    board[rand_x][rand_y] = True

    return GameObject(GameObjectType.FOOD, [rand_x / 10 - 1, rand_y / 10 - 1])

### Para enviar nossos dados da CPU para a GPU, precisamos requisitar um slot.

In [196]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Abaixo, enviamos todo o conteúdo da variável vertices.

Veja os parâmetros da função glBufferData [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [197]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

NameError: name 'vertices' is not defined

: 

### Associando variáveis do programa GLSL (Vertex Shaders) com nossos dados

Primeiro, definimos o byte inicial e o offset dos dados.

In [ ]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)

Em seguida, soliciamos à GPU a localização da variável "position" (que guarda coordenadas dos nossos vértices). Definimos essa variável no Vertex Shader.

In [ ]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

A partir da localização anterior, indicamos à GPU onde está o conteúdo (via posições stride/offset) para a variável position (aqui identificada na posição loc).

Outros parâmetros:

* Definimos que possui duas coordenadas
* Que cada coordenada é do tipo float (GL_FLOAT)
* Que não se deve normalizar a coordenada (False)

Mais detalhes: https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml

In [ ]:
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

In [ ]:
loc_color = glGetUniformLocation(program, "color")

### Nesse momento, exibimos a janela.


In [ ]:
glfw.show_window(window)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

A novidade agora é a função glDrawArrays().

Tal função recebe o tipo de primitiva (GL_TRIANGLES), o índice inicial do array de vértices (vamos mostrar todos os três vértices, por isso começamos com 0) e a quantidade de vértices ( len(vertices) ).

In [ ]:
def renderfps(framerate):
    global lastTime
              
#     currentTime = glfwGetTime(); 
    print(currentTime)

    if currentTime - lastTime >= 1.0 / framerate:
        lastTime = currentTime;
        return True
    return False


In [ ]:
def key_event(window,key,scancode,action,mods):
    if (action==1 or action==2):
        if key == 87: # tecla W
            # snake.update_direction(UP)    
            pass

        elif key == 83: # tecla S
            # snake.update_direction(DOWN)
            pass
            
        elif key == 65: # tecla A
            # snake.update_direction(LEFT)
            pass

        elif key == 68: # tecla D
            # snake.update_direction(RIGHT) 
            pass
       
    print('[key event] key=',key)
    print('[key event] scancode=',scancode)
    print('[key event] action=',action)
    print('[key event] mods=',mods)
    print('-------')
    

glfw.set_key_callback(window,key_event)

<function __main__.key_event(window, key, scancode, action, mods)>

In [ ]:
def mat_translate(x, y):
    return np.array([1.0, 0.0, 0.0, x,
                     0.0, 1.0, 0.0, y,
                     0.0, 0.0, 1.0, 0.0,
                     0.0, 0.0, 0.0, 1.0
                     ], np.float32)

mat_transform = mat_translate(0, 0) 

In [ ]:
while not glfw.window_should_close(window):

    global mat_transform
    
    glfw.poll_events() 
    
#     glPolygonMode(GL_FRONT_AND_BACK,GL_LINE) ## ative esse comando para enxergar os triângulos
    glClear(GL_COLOR_BUFFER_BIT) 
    glClearColor(1.0, 1.0, 1.0, 1.0)
    if i % 100 == 0:
        update() 
    i += 1
    
    mat_transform = mat_translate(pos_x, pos_y)
    
    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform)
    
    glUniform4f(loc_color, 0, 1, 0, 1.0) ### verde
    glDrawArrays(GL_TRIANGLE_STRIP, 0, 4) ### wormHead
    
    mat_food = np.eye(4)
    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_food)
    
    glUniform4f(loc_color, 1, 0, 0, 0.5) ### vermelho
    glDrawArrays(GL_TRIANGLE_STRIP, 4, 4) ### Food

    glfw.swap_buffers(window)

glfw.terminate()

[mouse event] button= 0
[mouse event] action= 1
[mouse event] mods= 0
-------
[key event] key= 68
[key event] scancode= 32
[key event] action= 1
[key event] mods= 0
-------
[mouse event] button= 0
[mouse event] action= 0
[mouse event] mods= 0
-------
[key event] key= 68
[key event] scancode= 32
[key event] action= 0
[key event] mods= 0
-------
[key event] key= 87
[key event] scancode= 17
[key event] action= 1
[key event] mods= 0
-------
[key event] key= 87
[key event] scancode= 17
[key event] action= 0
[key event] mods= 0
-------
[key event] key= 65
[key event] scancode= 30
[key event] action= 1
[key event] mods= 0
-------
[key event] key= 65
[key event] scancode= 30
[key event] action= 0
[key event] mods= 0
-------
[key event] key= 83
[key event] scancode= 31
[key event] action= 1
[key event] mods= 0
-------
[key event] key= 83
[key event] scancode= 31
[key event] action= 0
[key event] mods= 0
-------
[key event] key= 68
[key event] scancode= 32
[key event] action= 1
[key event] mods=